In [1]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
import yfinance as yf
import bs4 as bs
import pickle
import requests
from scipy import optimize

In [2]:
price = pd.read_csv('price.csv',index_col='Date')
sector = pd.read_csv('sector.csv',index_col=0).drop(index={'CDAY', 'HPE', 'PYPL','CARR', 'CSX', 'FTV', 'HWM', 'IR', 'OTIS'
                                                          ,'BRK.B', 'HIG','MRNA', 'OGN','ETSY', 'UA','BF.B', 'KHC', 'LW'
                                                          ,'REG','CEG','CTVA', 'DOW', 'WRK','DIS', 'FOXA', 'FOX'})

price = price.fillna(method = 'ffill')
price = price.fillna(method = 'bfill')

In [114]:
energy = sector[sector=='Energy'].dropna().index
IT = sector[sector=='Information Technology'].dropna().index
ind = sector[sector=='Industrials'].dropna().index
fi = sector[sector=='Financials'].dropna().index
hc = sector[sector=='Health Care'].dropna().index
cd = sector[sector=='Consumer Discretionary'].dropna().index
cs = sector[sector=='Consumer Staples'].dropna().index
re = sector[sector=='Real Estate'].dropna().index
ut = sector[sector=='Utilities'].dropna().index
ma = sector[sector=='Materials'].dropna().index
com = sector[sector=='Communication Services'].dropna().index

In [115]:
def find_coint_pairs(data, significance=0.01):
    n = data.shape[1]    
    score_matrix = np.zeros((n, n))
    pvalue_matrix = np.ones((n, n))
    keys = data.keys()
    pairs = []
    for i in range(n):
        for j in range(i+1, n):
            S1 = data[keys[i]]            
            S2 = data[keys[j]]
            result = ts.coint(S1, S2)
            score = result[0]
            pvalue = result[1]
            score_matrix[i, j] = score
            pvalue_matrix[i, j] = pvalue
            if pvalue < significance:
                pairs.append((keys[i], keys[j]))
    return score_matrix, pvalue_matrix, pairs

In [119]:
def sort(pairs):
    df = pd.DataFrame(index=pairs)
    df['pvalue']=0
    for i in range(len(pairs)):
        s1 = price[pairs[i][0]]
        s2 = price[pairs[i][1]]
        score, pvalue, _ = ts.coint(s1, s2)
        df.iloc[i,0] = pvalue
        
    return df.sort_values(by='pvalue')

df = sort(pairs)
    

In [121]:
pairs_IT = find_coint_pairs(price[IT], significance=0.01)
pairs_ind = find_coint_pairs(price[ind], significance=0.01)
pairs_fi = find_coint_pairs(price[fi], significance=0.01)
pairs_hc = find_coint_pairs(price[hc], significance=0.01)
pairs_cd = find_coint_pairs(price[cd], significance=0.01)
pairs_cs = find_coint_pairs(price[cs], significance=0.01)
pairs_re = find_coint_pairs(price[re], significance=0.01)
pairs_ut = find_coint_pairs(price[ut], significance=0.01)
pairs_ma = find_coint_pairs(price[ma], significance=0.01)
pairs_com = find_coint_pairs(price[com], significance=0.01)
pairs_energy = find_coint_pairs(price[energy], significance=0.01)

In [125]:
df_IT = sort(pairs_IT[2])
df_IT

,pvalue
"(ENPH, QCOM)",0.000004
"(QCOM, SEDG)",0.000041
"(ADP, MSI)",0.000285
"(ANSS, PAYC)",0.000303
"(ADBE, CRM)",0.000307
...,...
"(MCHP, MU)",0.008296
"(AMD, TYL)",0.008943
"(IBM, PTC)",0.008996
"(ACN, ENPH)",0.009072


In [126]:
df_ind = sort(pairs_ind[2])
df_ind

,pvalue
"(CTAS, CPRT)",0.000211
"(EFX, GNRC)",0.001614
"(DOV, J)",0.001816
"(CPRT, TDG)",0.002812
"(EFX, JCI)",0.003049
"(TDG, VRSK)",0.003574
"(ITW, NDSN)",0.003889
"(PWR, UPS)",0.004331
"(FBHS, GNRC)",0.004536
"(CHRW, CAT)",0.004785


In [127]:
df_fi = sort(pairs_fi[2])
df_fi

,pvalue
"(NTRS, RF)",0.000076
"(BAC, NTRS)",0.000274
"(SCHW, FITB)",0.000309
"(NTRS, TFC)",0.000438
"(SCHW, RJF)",0.000443
"(NTRS, PNC)",0.000506
"(FITB, RJF)",0.000877
"(NTRS, ZION)",0.001117
"(BK, CMA)",0.001168
"(ICE, WTW)",0.001346


In [128]:
df_hc = sort(pairs_hc[2])
df_hc

,pvalue
"(AMGN, HUM)",0.000115
"(RMD, ZTS)",0.000125
"(BAX, BDX)",0.000617
"(ABT, STE)",0.000780
"(CTLT, HOLX)",0.001453
"(LLY, MOH)",0.001641
"(CI, JNJ)",0.002355
"(RMD, STE)",0.002700
"(BSX, MRK)",0.003058
"(BDX, CAH)",0.003604


In [129]:
df_cd = sort(pairs_cd[2])
df_cd

,pvalue
"(KMX, GRMN)",0.000137
"(LOW, POOL)",0.000326
"(AMZN, BBY)",0.000331
"(GRMN, PHM)",0.000333
"(DHI, EBAY)",0.001086
"(KMX, DHI)",0.001753
"(GRMN, HD)",0.001801
"(DHI, POOL)",0.001829
"(APTV, LEN)",0.002139
"(NKE, POOL)",0.002412


In [130]:
df_cs = sort(pairs_cs[2])
df_cs

,pvalue
"(MDLZ, PG)",0.000536
"(HRL, WBA)",0.004842
"(CL, MNST)",0.005201
"(HRL, MKC)",0.007247


In [131]:
df_re = sort(pairs_re[2])
df_re

,pvalue


In [132]:
df_ut = sort(pairs_ut[2])
df_ut

,pvalue
"(ES, WEC)",0.000318
"(ED, NI)",0.001149
"(ES, XEL)",0.003632
"(EIX, PPL)",0.004537
"(D, ES)",0.005487
"(CMS, D)",0.005518
"(DTE, PEG)",0.007461
"(AEP, SRE)",0.008312


In [133]:
df_ma = sort(pairs_ma[2])
df_ma

,pvalue
"(AMCR, VMC)",0.000361
"(APD, BLL)",0.000954
"(AMCR, MLM)",0.002444
"(IFF, PKG)",0.003531
"(IFF, SHW)",0.007560
"(IFF, VMC)",0.009007


In [134]:
df_com = sort(pairs_com[2])
df_com

,pvalue
"(GOOGL, GOOG)",0.001741
"(ATVI, EA)",0.003406
"(CHTR, FB)",0.006498


In [135]:
df_energy = sort(pairs_energy[2])
df_energy

,pvalue
"(FANG, EOG)",0.000124


In [40]:
sector.value_counts()

Sectors               
Information Technology    74
Industrials               73
Financials                66
Health Care               65
Consumer Discretionary    60
Consumer Staples          32
Real Estate               30
Utilities                 29
Materials                 28
Communication Services    26
Energy                    21
dtype: int64